In [ ]:
# =========================================


In [ ]:
# =========================================
# 🧠 模型加载与 TTA 推理
# =========================================

# 1️⃣ 加载模型结构
model = MyDualStreamModel(
    "convnext_tiny",
    pretrained=False,
    freeze_ratio=config["freeze_ratio"],
    weights_dict=weights,
)
model = model.to(device)
model = model.to(memory_format=torch.channels_last)

# 2️⃣ 设置模型目录（根据运行环境自动切换）
if socket.gethostname() == "hao-2":
    model_dir = history_DIR
else:
    model_dir = DIRS["model"]

# 3️⃣ 执行 TTA（Test-Time Augmentation）推理
tta_preds = []

for name, tform in tta_transforms.items():
    print(f"\n🚀 Running TTA: {name}")
    
    transform = tform
    df_pred5 = predict_ensemble_df(
        df_test=df_test,
        transform=transform,
        model=model,
        model_target_cols=model_target_cols,
        model_dir=model_dir,
        device=device,
    )

    print(df_pred5.head())  # ✅ 可选：仅展示前几行结果
    tta_preds.append(df_pred5[target_cols].values)

# 4️⃣ 汇总 TTA 结果并计算平均预测
print("\n📊 聚合 TTA 结果...")
mean_preds = np.mean(tta_preds, axis=0)

# 5️⃣ 生成最终预测 DataFrame
df_pred_final = df_pred5.copy()
df_pred_final[target_cols] = mean_preds
show_df_info(df_pred_final, "df_pred_final")
